## λ calculus

Here  is a Python notebook to play with the lamdba calculus and  evaluate lambda expressions of the form `λvar1(exp1) λvar2(exp2) ...`. If you don't know Python you can safely ignore the Ptyhon code and skip below to where we actually talk about the λ calculus itself.

To better fit with python there are two main differences:

* Instead of writing `λvar.exp` we write `λvar(exp)`

* Instead of simply concatenating two expressions `exp1 exp2` we use the `*` operator and write `exp1 * exp2`. We can also use `exp1, exp2` if they are inside a function call or a variable binding parenthesis.

* To reduce an expression `exp`, use `exp.reduce()`

* Since Python does not allow us to override the default `0` and `1` we use `_0` for `λx(y(y))` and `_1` for `λx(y(x))`. 




## Python code (can skip at first read)

If you don't know Python feel free to skip ahead to the part where we play with the $\lambda$ calculus itself.


In [43]:
import operator ,functools

class Lambdaexp:
    """Lambda expressions base class"""
    
    counter = 0
    
    def __init__(self):
        self.mykey = {}
    
    def apply(self,other): 
        """Apply expression on an argument"""
        return self*other
        
    def _reduce(self,maxlevel=100): 
        """Reduce expression"""
        return self
    
    def replace(self,old,new): 
        """Replace all occurences of old with new"""
        raise NotImplemented

    
    def bounded(self): 
        """Set of bounded variables inside expression"""
        return set()

    def asstring(self, m,pretty=False):
        """Represent self as a string mapping bounded variables to particular numbers."""
        raise NotImplemented

    #------------------------------------------------------------------------------#
    # Ignore this code  in first read: Python specific details

    lambdanames = {}
    reducedstrings = {}
   
    def reduce(self,maxlevel=100):
        if not maxlevel: return self
        #m = {b:b for b in self.bounded() }
        #t = Lambdaexp.reducedstrings.get((self.asstring(m),maxlevel),None)
        #if t: return t
        return self._reduce(maxlevel)
        #k = t.asstring(m)
        #for i in range(maxlevel+1):  
        #    Lambdaexp.reducedstrings[(k,i)] = t
        #return t
            
            
        
    def __mul__(self,other):
        """Use * for combining."""
        return Combinedexp(self,other) if other else self
    
    def __call__(self,*args): 
        """Use function call for application"""
        return functools.reduce(operator.mul,args,self)
    
    def _key(self,maxlevel=100): 
        #if maxlevel not in self.mykey: 
        return self.reduce(maxlevel).__repr__()
            # for i in range(maxlevel+1): self.mykey[i] = s 
        # return self.mykey[maxlevel]

    def __eq__(self,other): return self._key()==other._key() if isinstance(other,Lambdaexp) else False
    def __hash__(self): return hash(self._key())
    
        
        
    def __repr__(self,pretty=False):
        B = sorted(self.bounded())
        m ={}
        for v in B: m[v] = len(m)
        return self.asstring(m,pretty)
    
    
    def _repr_pretty_(self, p, cycle):
        if cycle: p.text( self._repr())
        p.text( self.reduce().__repr__(True))     
        
    def addconst(self,srep):
        """Return either exp.string or replaced with a keyword if it's in table."""
        if self in Lambdaexp.lambdanames:  return blue(Lambdaexp.lambdanames[self])
        return srep

      #------------------------------------------------------------------------------#


    
    
 

In [44]:
#-------------------------------------------------#
# Utility functions: print color
def bold(s,justify=0):
    return "\x1b[1m"+s.ljust(justify)+"\x1b[21m"

def underline(s,justify=0):
    return "\x1b[4m"+s.ljust(justify)+"\x1b[24m"

def red(s,justify=0):
    return  "\x1b[31m"+s.ljust(justify)+"\x1b[0m"


def green(s,justify=0):
    return  "\x1b[32m"+s.ljust(justify)+"\x1b[0m"


def blue(s,justify=0):
    return  "\x1b[34m"+s.ljust(justify)+"\x1b[0m"
#--------------------------------------------------#

In [45]:
  
    
    

        
class Applicableexp(Lambdaexp):
    """Lambda expression that can be applied"""
    
    def __init__(self,exp,name,newarg=-1):
        Lambdaexp.counter += 1
        self.arg = newarg if newarg>=0 else Lambdaexp.counter
        self.inner = exp.replace(name,Boundvar(self.arg))
        super().__init__()    
    
    def apply(self,other): 
        return self.inner.replace(self.arg,other)
    
    def replace(self,old,new): 
        if self.arg==old: raise Exception(f"calling {self}.replace({old},{new})") # maybe should return self
        return Applicableexp(self.inner.replace(old,new),self.arg)
    
    def bounded(self): return self.inner.bounded()|{self.arg}
    
    def _reduce(self,maxlevel=100):
        inner = self.inner.reduce(maxlevel-1)
        return Applicableexp(inner,self.arg,self.arg)
    
    def asstring(self, m,pretty=False):
        if not pretty: return "λ"+Boundvar(self.arg).asstring(m,False)+".("+self.inner.asstring(m)+")"
        return self.addconst(green("λ")+Boundvar(self.arg).asstring(m,True)+".("+self.inner.asstring(m,True)+")")
        
        
 
  

In [46]:
class Binder:
    """Binds a variable name in a lambda expression"""

    def __init__(self,name): self.name = name
    
    def bindexp(self,exp):
        return Applicableexp(exp,self.name)
    
    #------------------------------------------------------------------------------#
    # Ignore this code  in first read: Python specific details

    
    def __call__(self,*args):
        exp = functools.reduce(operator.mul,args[1:],args[0])
        return self.bindexp(exp)
    #------------------------------------------------------------------------------#

In [47]:
class Boundvar(Lambdaexp):
    """Bounded variable"""
    def __init__(self,arg): 
        self.myid = arg
        super().__init__()
        
    def replace(self,argnum,exp): return exp if argnum==self.myid else self
    
    def bounded(self): return { self.myid }

    def asstring(self, m,pretty=False):
        arg = m.get(self.myid,self.myid) 
        return chr(ord('α')+arg)
    

class Unboundvar(Lambdaexp):
    """Unbounded (free) variable."""
    def __init__(self,name): 
        self.name = name
        super().__init__()
        
    def replace(self,name,arg): return arg if name==self.name else self
    
    def asstring(self, m,pretty=False):
        return self.addconst(self.name) if pretty else self.name

   
    
class Combinedexp(Lambdaexp):
    """Combined expression of two expressions."""
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
        super().__init__()
    
    def replace(self,arg,exp): 
        return Combinedexp(self.exp1.replace(arg,exp),self.exp2.replace(arg,exp))
    
    def bounded(self): return self.exp1.bounded()|self.exp2.bounded()
    
    
    def _reduce(self,maxlevel=100):
        if not maxlevel: return self
        e1 = self.exp1.reduce(maxlevel-1)
        if isinstance(e1,Applicableexp): 
            return  e1.apply(self.exp2).reduce(maxlevel-1)
        return Combinedexp(e1,self.exp2)
    
    def asstring(self, m,pretty=False):
        s = f"({self.exp1.asstring(m,False)} {self.exp2.asstring(m,False)})"
        if not pretty: return s
        return f"({self.exp1.asstring(m,True)} {self.exp2.asstring(m,True)})"


### Initalization 

The above is all the code for implementing the λ calculus. We now add some convenient global variables: 
λa .... λz and a ... z for variables, and 0 and 1.

In [48]:
Lambdaexp.lambdanames  = {}
import string

def initids(g):
    """Set up parameters a...z and correpsonding Binder objects λa..λz"""
    ids = list(string.ascii_lowercase)
    for name in ids:
        g[name] = Unboundvar(name)
        g["λ"+name] = Binder(name)
        Lambdaexp.lambdanames[g[name]] = name

In [49]:
initids(globals())

In [50]:
# testing...
λy(y)

λα.(α)

In [51]:
def setconstants(g,consts):
    """Set up constants for easier typing and printing."""
   
    for name in consts:
        Lambdaexp.lambdanames[consts[name]] = name
        if name[0].isalpha(): 
            g[name]=consts[name]
            
        else: # Numeric constants such as 0 and 1 are replaced by _0 and _1
            g["_"+name] = consts[name]
    
setconstants(globals(),{"1" : λx(λy(x)) , "0" : λx(λy(y))  })

def register(g,*args):
    for name in args:
        Lambdaexp.lambdanames[g[name]] = name

In [52]:
# testing
λa(λz(a)).__repr__()

'λα.(λβ.(α))'

## λ calculus playground

We can now start playing with the λ calculus

If you want to use the λ character you can copy paste it from here:  λ

Let's start with the function λx,y.y, also known as 0 

In [53]:
λa(λb(b))

0

Our string representation recognizes that this is the 0 function and so "pretty prints" it. To see the underlying λ expression you can use `__repr__()`

In [54]:
λa(λb(b)).__repr__()

'λα.(λβ.(β))'

Let's check that `_0` and `_1` behave as expected

In [55]:
_1(a,b)

a

In [58]:
_0(a,b)

b

In [59]:
_1

1

In [60]:
_1(_0)

λα.(0)

In [62]:
_1.__repr__()

'λα.(λβ.(α))'

Here is an exercise:

__Question:__ Suppose that $F = \lambda f. (\lambda x. (f x)f)$, $1 = \lambda x.(\lambda y.x)$ and $0=\lambda x.(\lambda y.y)$.
What is $F \; 1\; 0$?

__a.__ $1$

__b.__ $0$

__c.__ $\lambda x.1$

__d.__ $\lambda x.0$

Let's evaluate the answer

In [64]:
F=λf(λx((f*x)*f))
F

λα.(λβ.(((α β) α)))

In [70]:
F(_1)

ID

In [71]:
F(_1,_0)

0

In [72]:
ID = λa(a)
register(globals(),"ID")

### Some useful functions

Let us now add some of the basic functions in the λ calculus

In [94]:
NIL= λf(_1)
PAIR =λx(λy(λf(f*x*y)))
ISEMPTY= λp(p *(λx(λy(_0))))
HEAD = λp(p(_1))
TAIL  =λp(p * _0)

register(globals(),"NIL", "PAIR")

And test them out

In [95]:
ISEMPTY(NIL)

1

In [96]:
PAIR

PAIR

In [97]:
P=PAIR(_0,_1)

In [98]:
HEAD(P)

0

In [99]:
TAIL(P)

1

We can make lists of bits as follows:

In [100]:
def makelist(*L):
    """Construct a λ list of _0's and _1's."""
    if not L: return NIL
    h = _1 if L[0]   else _0
    return PAIR(h,makelist(*L[1:]))

In [101]:
L=makelist(1,0,1)
L

λα.(((α 1) ((PAIR 0) ((PAIR 1) NIL))))

In [102]:
HEAD(L)

1

In [103]:
TAIL(L)

λα.(((α 0) ((PAIR 1) NIL)))

In [104]:
HEAD(TAIL(L))

0

In [106]:
HEAD(TAIL(TAIL(L)))

1

## Recursion and the Y combinator

Now let's try something a little more interesting

The $XOR$ function on arbitrary length lists can be defined recursively as follows:

$$XOR(\ell) = \begin{cases} 0 & ISEMPTY(\ell) \\ XOR_2(HEAD(\ell),XOR(TAIL(\ell)) & \text{otherwise} \end{cases}$$

where $\ell$ is a list and $XOR_2$ is the XOR on two bits.

The following is the definition of XOR on 2 bits - can you see why?

In [107]:
XOR2 = λa(λb(a * (b*_0*_1) * b ))

In [108]:
XOR2(_1,_1)

0

Using the fact that $IF(a,b,c)=  (a b c)$ in the λ calculus,  which we write as `(a * b * c)` here, we see the XOR is the function `f` that is a fixed poing of the following function `RECXOR`

In [109]:
RECXOR = λf(λl( (ISEMPTY*l) * _0 * (XOR2 * (HEAD * l) * (f * (TAIL * l)))  ))

We can now define the **Y Combinator** which is an operator that finds fixed points for every λ expression.

In [110]:
Y = λf((λx(f * (x * x))) (λy(f * (y * y))))

Let's try it out:

In [111]:
XOR = Y(RECXOR)

In [112]:
XOR(PAIR(_1,NIL))

1

In [113]:
XOR(PAIR(_1,PAIR(_1,NIL)))

0

And now let's see how it works on longer lists

In [114]:
L = makelist(1,0,0,1,1,0,1,0,1)

In [115]:
XOR(L)

1

In [116]:
XOR(makelist(1,0,1,1,1,0,1,0,1))

0